In [250]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json

def clean_dataframe():

    def load_file():
        
        try:
            df = pd.read_csv('mental_health.csv')
            return df
        except Exception as e:
            print(f'Erro ao carregar o dataframe "mental_healh":{e}')


    def columns_mapping(df: pd.DataFrame):
        columns_mapping = {}
        # Nicknanes paras as colunas em arquivo json
        with open('json_files/variable_mapping.json', 'r') as file:
            columns_mapping = json.load(file)

        # Renomeia o nome das colunas para nomes mais curtos
        df.rename(columns=columns_mapping, inplace=True)

        return df

    def age_gender_featuring(df: pd.DataFrame):
        
        # Ajeitando a idade
        try:
            # Converte todos os valores para numéricos e substitui os 'erros' para nan
            df['idade'] = pd.to_numeric(df['idade'], errors='coerce')

            condition = (df['idade'] < 18) | (df['idade'] > 100)
            df['idade'] = np.where(condition, np.nan, df['idade'])

        except KeyError:
            print(f"Erro: A coluna 'idade' não existe no DataFrame.")
        
        except ValueError:
            print(f"Erro: Dados inválidos na coluna idade")

        # Ajeitando o genero
        try:    
            gender_mapping = {}
            with open('json_files/gender_mapping.json', 'r') as file:
                gender_mapping = json.load(file)

            df['genero'] = df['genero'].astype(str)
            df['genero'] = df['genero'].str.replace(r'\d+', '')
            df['genero'] = df['genero'].str.lower().map(gender_mapping).fillna('Other')
        
        except KeyError:
            print(f"Erro: A coluna genero não existe no DataFrame.")
        
        except ValueError:
            print(f"Erro: Dados inválidos na coluna gender")

        
        # Gráfico
        df_ = df.dropna(subset=['idade'])

        sns.set(style='white', font_scale=1.0)

        # Parameters
        title = 'Dist. Idade por Gênero'
        x_axis = 'idade'
        x_label = 'Idade'
        hue = 'genero'
        
        custom_pallete = {'M': 'indigo', 'F': 'limegreen', 'Other': 'thistle'}
        sns.histplot(data=df_, x=x_axis, hue=hue, kde=True, palette=custom_pallete)

        plt.title(title)
        plt.xlabel(x_label)
        plt.ylabel('Contagem')

        plt.close()

        os.makedirs('images', exist_ok=True)
        save_path = 'images/dist_idade_sexo.png'
        plt.savefig(save_path, dpi=400)

        return df

    def edit_dataframe(df: pd.DataFrame):

        # Edição dos valores para melhor interpretação das colunas

        # Alguns padrões que se repetem
        dict_1 = {
            '1.0': 'Yes',
            'True': 'Yes', 
            'False': 'No',
            '0.0': 'No'
            }

        # Colunas
        df['emp_tec'] = df['emp_tec'].map(dict_1)

        df['papel_tec'] = df['papel_tec'].map(dict_1)

        df['discussao_saude_mental_emp'] = df['discussao_saude_mental_emp'].map(dict_1)

        
        index_rename = {'Not eligible for coverage / NA': 'Not eligible'}
        df.replace({'beneficios_mental': index_rename}, inplace=True)
        
        df['conforto_discutir_colegas'] = df['conforto_discutir_colegas'].map({
        'Maybe': 'Maybe',
        'Yes': 'Yes', 
        'No': 'No'
        })    

        df['discussao_saude_mental_colegas'] = df['discussao_saude_mental_colegas'].map({
        '1': 'Yes', 
        '0': 'No',
        'No': 'No',
        'Yes': 'Yes',
        'True': 'Yes',
        'False': 'No',
        '0.0': 'No',
        '1.0': 'Yes'
        })

        df['discussao_saude_mental_colegas_outro'] = df['discussao_saude_mental_colegas_outro'].map({
        '1': 'Yes', 
        '0': 'No',
        'True': 'Yes',
        'False': 'No',
        '0.0': 'No',
        '1.0': 'Yes'
        })

        df['importancia_saude_fisica_emp'] = pd.to_numeric(df['importancia_saude_fisica_emp'], errors='coerce')

        df['importancia_saude_mental_emp'] = pd.to_numeric(df['importancia_saude_mental_emp'], errors='coerce')

        df['emp_tec_anterior'] = df['emp_tec_anterior'].map({'True': 1, 'False': 0})
        df['emp_tec_anterior'] = pd.to_numeric(df['emp_tec_anterior'], errors='coerce')

        # Map the values according to the specifications
        df['disturbio_saude_mental_atual'] = df['disturbio_saude_mental_atual'].map({
            "Don't Know": "I don't know",
            '0': 'No',
            'No': 'No',
            '1': 'Yes',
            '4': 'Yes',
            '7': 'Yes',
            '5': 'Yes',
            '8': 'Yes',
            '2': 'Yes',
            '3': 'Yes',
            'Yes': 'Yes',
            'Possibly': 'Maybe'
        })

        df['tratamento_profissional_mental'] = df['tratamento_profissional_mental'].map({
        "Don't Know": "I don't know",
        '0': 'No',
        'No': 'No',
        'FALSE': 'No',
        '1': 'Yes',
        'TRUE': 'Yes',
        'Yes': 'Yes',
        'Possibly': 'Maybe'
        })

        df['historico_familiar_mental'] = df['historico_familiar_mental'].map({
        "I don't Know": "I don't know",
        '0': 'No',
        'No': 'No',
        'FALSE': 'No',
        '1': 'Yes',
        'TRUE': 'Yes',
        'Yes': 'Yes',
        'Possibly': 'Maybe'
        })


        df['interferencia_sem_tratamento_eficaz'] = df['interferencia_sem_tratamento_eficaz'].map(
        {"1":"Yes",
        "Sometimes":"Sometimes",
        "Often": "Often",
        "Not applicable to me": "Not applicable to me",
        "Rarely": "Rarely",
        "Never": "Never",
        })

        # Map the values according to the specifications
        df['disposto_compartilhar_amigos_familiares'] = df['disposto_compartilhar_amigos_familiares'].map({
            "Sometimes": "Sometimes",
            "Maybe": "Maybe",
            '0': 'No',
            'No': 'No',
            '1': 'Yes',
            '5': 'Yes',
            '4': 'Yes',
            '10': 'Yes',
            '8': 'Yes',
            '3': 'Yes',
            '2': 'Yes',
            '9': 'Yes',
            '7': 'Yes',
            '1': 'Yes',
            'Yes': 'Yes',
        })

        # Map the values according to the specifications
        df['disposto_apresentar_saude_entrevista'] = df['disposto_apresentar_saude_entrevista'].map({
            "Yes": "Yes",
            "Maybe": "Maybe",
            '0': 'No',
            'No': 'No',
            '8': 'Yes',
            '7': 'Yes',
            '3': 'Yes',
            '10': 'Yes',
            '6': 'Yes',
            '1': 'Yes',
            '5': 'Yes',
            '9': 'Yes',
            'Often': 'Often'
        })

        df['identificado_abertamente_mental'] = df['identificado_abertamente_mental'].map({
        'TRUE': 'Yes',
        'FALSE': 'No',
        '1': 'Yes',
        '0': 'No'
        })

        df['traria_saude_mental_entrevista'] = df['traria_saude_mental_entrevista'].map({
        'No': 'No',
        'Maybe': 'Maybe',
        'Yes': 'Yes'
        })

        return df
    
    # Chama as funções
    df = load_file()
    df = columns_mapping(df)
    df = age_gender_featuring(df)
    df = edit_dataframe(df)

    # Salva o datafreame modificado
    os.makedirs('../../data_ready_to_model', exist_ok=True)
    save_path = '../../data_ready_to_model/mental_health.csv'
    df.to_csv(save_path, index=False)
    


In [251]:
clean_dataframe()

<Figure size 640x480 with 0 Axes>

## 1 Feature Engineering

### Idade e Genero

## Outras colunas

In [171]:
# Conversão para valoes inteiros
df['emp_tec'] = df['emp_tec'].map({
    '1.0': 'Yes',
    'True': 'Yes', 
    'False': 'No',
    '0.0': 'No'
    })
df['emp_tec'] = pd.to_numeric(df['emp_tec'], errors='coerce')
df['emp_tec'].unique()

array([nan,  1.,  0.])

In [172]:
df['papel_tec'] = df['papel_tec'].map({
    '1.0': 'Yes',
    'True': 'Yes', 
    'False': 'No',
    '0.0': 'No'})

array([nan,  1.,  0.])

In [174]:
index_rename = {'Not eligible for coverage / NA': 'Not eligible'}
df.replace({'beneficios_mental': index_rename}, inplace=True)

In [177]:
# Conversão para valoes inteiros
df['discussao_saude_mental_emp'] = df['discussao_saude_mental_emp'].map({
    '1.0': 'Yes',
    'True': 'Yes', 
    'False': 'No',
    '0.0': 'No'})
df['discussao_saude_mental_emp'].unique()

array([nan,  0.,  1.])

In [220]:
df['conforto_discutir_colegas'] = df['conforto_discutir_colegas'].map({
    'Maybe': 'Maybe',
    'Yes': 'Yes', 
    'No': 'No'
    })

In [221]:
df['conforto_discutir_colegas'].unique()

array(['Yes', 'Maybe', nan, 'No'], dtype=object)

In [178]:
df['discussao_saude_mental_colegas'] = df['discussao_saude_mental_colegas'].map({
    '1': 'Yes', 
    '0': 'No',
    'No': 'No',
    'Yes': 'Yes',
    'True': 'Yes',
    'False': 'No',
    '0.0': 'No',
    '1.0': 'Yes'
    })
df['discussao_saude_mental_colegas'].isna().value_counts()

discussao_saude_mental_colegas
True     1689
False     147
Name: count, dtype: int64

In [205]:
df['discussao_saude_mental_colegas_outro'].value_counts()

df['discussao_saude_mental_colegas_outro'] = df['discussao_saude_mental_colegas_outro'].map({
    '1': 'Yes', 
    '0': 'No',
    'True': 'Yes',
    'False': 'No',
    '0.0': 'No',
    '1.0': 'Yes'
    })

discussao_saude_mental_colegas_outro
0                                                                                                                                                                                                                                                                       597
1                                                                                                                                                                                                                                                                       545
False                                                                                                                                                                                                                                                                   162
True                                                                                                                                                           

In [179]:
df['importancia_saude_fisica_emp'] = pd.to_numeric(df['importancia_saude_fisica_emp'], errors='coerce')
df['importancia_saude_fisica_emp'].isna().value_counts()

importancia_saude_fisica_emp
False    1563
True      273
Name: count, dtype: int64

In [180]:
df['importancia_saude_mental_emp'] = pd.to_numeric(df['importancia_saude_mental_emp'], errors='coerce')
df['importancia_saude_mental_emp'].isna().value_counts()

importancia_saude_mental_emp
False    1576
True      260
Name: count, dtype: int64

In [181]:
df['emp_tec_anterior'] = df['emp_tec_anterior'].map({'True': 1, 'False': 0})
df['emp_tec_anterior'] = pd.to_numeric(df['emp_tec_anterior'], errors='coerce')
df['emp_tec_anterior'].unique()

array([nan,  0.,  1.])

In [182]:
# Map the values according to the specifications
df['disturbio_saude_mental_atual'] = df['disturbio_saude_mental_atual'].map({
    "Don't Know": "I don't know",
    '0': 'No',
    'No': 'No',
    '1': 'Yes',
    '4': 'Yes',
    '7': 'Yes',
    '5': 'Yes',
    '8': 'Yes',
    '2': 'Yes',
    '3': 'Yes',
    'Yes': 'Yes',
    'Possibly': 'Maybe'
})

print(df['disturbio_saude_mental_atual'].unique())
print(df['disturbio_saude_mental_atual'].value_counts())

['Maybe' nan "I don't know" 'Yes' 'No']
disturbio_saude_mental_atual
Maybe           370
I don't know    162
Yes              13
No                2
Name: count, dtype: int64


In [184]:
df['tratamento_profissional_mental'] = df['tratamento_profissional_mental'].map({
    "Don't Know": "I don't know",
    '0': 'No',
    'No': 'No',
    'FALSE': 'No',
    '1': 'Yes',
    'TRUE': 'Yes',
    'Yes': 'Yes',
    'Possibly': 'Maybe'
})

In [185]:
df['tratamento_profissional_mental'].value_counts()

tratamento_profissional_mental
Yes             1068
No               751
Maybe              4
I don't know       1
Name: count, dtype: int64

In [186]:
df['historico_familiar_mental'] = df['historico_familiar_mental'].map({
    "I don't Know": "I don't know",
    '0': 'No',
    'No': 'No',
    'FALSE': 'No',
    '1': 'Yes',
    'TRUE': 'Yes',
    'Yes': 'Yes',
    'Possibly': 'Maybe'
})

In [187]:
df['historico_familiar_mental'].value_counts()

historico_familiar_mental
Yes    817
No     593
Name: count, dtype: int64

In [188]:
df['interferencia_sem_tratamento_eficaz'] = df['interferencia_sem_tratamento_eficaz'].map(
    {"1":"Yes",
     "Sometimes":"Sometimes",
     "Often": "Often",
     "Not applicable to me": "Not applicable to me",
     "Rarely": "Rarely",
     "Never": "Never",
     })

In [189]:
df['interferencia_sem_tratamento_eficaz'].isna().value_counts()

interferencia_sem_tratamento_eficaz
False    1836
Name: count, dtype: int64

In [190]:
# Map the values according to the specifications
df['disposto_compartilhar_amigos_familiares'] = df['disposto_compartilhar_amigos_familiares'].map({
    "Sometimes": "Sometimes",
    "Maybe": "Maybe",
    '0': 'No',
    'No': 'No',
    '1': 'Yes',
    '5': 'Yes',
    '4': 'Yes',
    '10': 'Yes',
    '8': 'Yes',
    '3': 'Yes',
    '2': 'Yes',
    '9': 'Yes',
    '7': 'Yes',
    '1': 'Yes',
    'Yes': 'Yes',
})

In [191]:
# Map the values according to the specifications
df['disposto_apresentar_saude_entrevista'] = df['disposto_apresentar_saude_entrevista'].map({
    "Yes": "Yes",
    "Maybe": "Maybe",
    '0': 'No',
    'No': 'No',
    '8': 'Yes',
    '7': 'Yes',
    '3': 'Yes',
    '10': 'Yes',
    '6': 'Yes',
    '1': 'Yes',
    '5': 'Yes',
    '9': 'Yes',
    'Often': 'Often'
})

In [192]:
df['identificado_abertamente_mental'] = df['identificado_abertamente_mental'].map({
    'TRUE': 'Yes',
    'FALSE': 'No',
    '1': 'Yes',
    '0': 'No'
    })
df['identificado_abertamente_mental'].unique()

array([ 0.,  1., nan])

In [193]:
df['identificado_abertamente_mental'].value_counts()

identificado_abertamente_mental
0.0    1588
1.0     221
Name: count, dtype: int64

In [197]:
df['traria_saude_mental_entrevista'] = df['traria_saude_mental_entrevista'].map({
    'No': 'No',
    'Maybe': 'Maybe',
    'Yes': 'Yes'
    })

In [198]:
df['traria_saude_mental_entrevista'].value_counts()

traria_saude_mental_entrevista
No       1244
Maybe     469
Yes       104
Name: count, dtype: int64

In [202]:
for i in df.columns:
    print(i)
    print(df[i].unique())
    print(df[i].isna().value_counts())
    print('\n')

autonomo
['0' '1' 'False' 'True']
autonomo
False    1836
Name: count, dtype: int64


num_func
['100-500' '6-25' 'More than 1000' nan '26-100' '1-5' '500-1000']
num_func
False    1577
True      259
Name: count, dtype: int64


emp_tec
[nan  1.  0.]
emp_tec
True     1532
False     304
Name: count, dtype: int64


papel_tec
[nan  1.  0.]
papel_tec
True     1532
False     304
Name: count, dtype: int64


beneficios_mental
['No' 'Yes' "I don't know" nan 'Not eligible']
beneficios_mental
False    1577
True      259
Name: count, dtype: int64


conhecimento_opcoes_mental
['Yes' 'No' nan]
conhecimento_opcoes_mental
False    1414
True      422
Name: count, dtype: int64


discussao_formal_mental
['No' "I don't know" nan 'Yes']
discussao_formal_mental
False    1577
True      259
Name: count, dtype: int64


recursos_aprendizado_mental
["I don't know" 'No' nan 'Yes']
recursos_aprendizado_mental
False    1577
True      259
Name: count, dtype: int64


anonimato_protegido
["I don't know" 'Yes' nan 'No']
a

In [203]:
for i in df_teste.columns:
    print(i)
    print(df_teste[i].unique())
    print(df_teste[i].isna().value_counts())
    print('\n')

Você é autônomo?
['0' '1' 'False' 'True']
Você é autônomo?
False    1836
Name: count, dtype: int64


Quantos funcionários sua empresa ou organização possui?
['100-500' '6-25' 'More than 1000' nan '26-100' '1-5' '500-1000']
Quantos funcionários sua empresa ou organização possui?
False    1577
True      259
Name: count, dtype: int64


Seu empregador é principalmente uma empresa/organização de tecnologia?
['1.0' nan '0.0' 'True' 'False']
Seu empregador é principalmente uma empresa/organização de tecnologia?
False    1577
True      259
Name: count, dtype: int64


O seu papel principal na sua empresa está relacionado à tecnologia/TI?
['1.0' nan '0.0' 'True' 'False']
O seu papel principal na sua empresa está relacionado à tecnologia/TI?
False    1577
True      259
Name: count, dtype: int64


O seu empregador oferece benefícios de saúde mental como parte da cobertura de saúde?
['No' 'Yes' "I don't know" nan 'Not eligible for coverage / NA']
O seu empregador oferece benefícios de saúde mental 

### Feature Importance

In [195]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import shap
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_predict

# # Assuming you have a dataframe named 'df' with features and a target column
# # 'X' contains the feature columns

# target = 'Você atualmente tem um distúrbio de saúde mental?'
# X = df.drop(columns=target, axis=1)

# # 'y' contains the target column
# y = df[target]

# # Create a Random Forest classifier
# rf = RandomForestClassifier()

# # Perform cross-validated predictions
# y_pred = cross_val_predict(rf, X, y, cv=5, n_jobs=-1)

# # Fit the classifier to the entire dataset
# rf.fit(X, y)

# # Initialize a SHAP explainer
# explainer = shap.Explainer(rf)

# # Calculate SHAP values for all features based on the entire dataset
# shap_values = explainer(X)

# # Calculate feature importances based on mean absolute SHAP values
# feature_importances = pd.DataFrame(
#     list(zip(X.columns, shap_values.abs().mean(0))),
#     columns=['Feature', 'Importance']
# )

# # Sort the dataframe by importance in descending order
# feature_importances = feature_importances.sort_values('Importance', ascending=False)

# # Print the feature importance ranking
# print(feature_importances)

# # Plot feature importances
# plt.figure(figsize=(10, 6))
# plt.bar(feature_importances['Feature'], feature_importances['Importance'])
# plt.xticks(rotation=90)
# plt.xlabel('Features')
# plt.ylabel('Importance')
# plt.title('Feature Importance (SHAP)')
# plt.tight_layout()
# plt.show()
